### Configurando o collab para rodar pyspark

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
!ls

sample_data  spark-3.1.1-bin-hadoop3.2	spark-3.1.1-bin-hadoop3.2.tgz


In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

### Montando GoogleDrive

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#### Explorando os dados

In [6]:
df_modelo = spark.read.csv('/content/gdrive/MyDrive/DataMasterMLOps/CodigosCase/BaseModelo/casas.csv', header=True)

In [23]:
df_modelo.count()

1460

In [25]:
df_modelo.show(5)

+-------+----+-------+------+
|tamanho| ano|garagem| preco|
+-------+----+-------+------+
|  159.0|2003|      2|208500|
|  117.0|1976|      2|181500|
|  166.0|2001|      2|223500|
|  160.0|1915|      3|140000|
|  204.0|2000|      3|250000|
+-------+----+-------+------+
only showing top 5 rows



In [8]:
df_sample_submission = spark.read.csv('/content/gdrive/MyDrive/DataMasterMLOps/CodigosCase/BaseKaggle/sample_submission.csv', header=True)

In [9]:
df_sample_submission.show(5)

+----+----------------+
|  Id|       SalePrice|
+----+----------------+
|1461|  169277.0524984|
|1462|187758.393988768|
|1463|183583.683569555|
|1464| 179317.47751083|
|1465|150730.079976501|
+----+----------------+
only showing top 5 rows



In [10]:
df_sample_submission.count()

1459

In [11]:
df_test = spark.read.csv('/content/gdrive/MyDrive/DataMasterMLOps/CodigosCase/BaseKaggle/test.csv', header=True)

In [ ]:
df_test.show(5)

+----+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+
|  Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgTy

In [12]:
df_test.count()

1459

In [13]:
df_train = spark.read.csv('/content/gdrive/MyDrive/DataMasterMLOps/CodigosCase/BaseKaggle/train.csv', header=True)

In [14]:
df_train.show(5)

+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+
| Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition

In [15]:
df_train.count()

1460

Variaveis que são usadas para o meu modelo simples:

*   garagem   = GarageCars: Size of garage in car capacity = garagem
*   SalePrice = preço (target)
*   YearBuilt: Original construction date = ano
*   LotArea: Lot size in square feet = array_equal

A tabela Sample, foi um modelo que rodou e gerou essas Informações, train é para treino e test é para testar o modelo.













In [29]:
df = df_train.select('GarageCars','SalePrice','YearBuilt','LotArea')
df.show()

+----------+---------+---------+-------+
|GarageCars|SalePrice|YearBuilt|LotArea|
+----------+---------+---------+-------+
|         2|   208500|     2003|   8450|
|         2|   181500|     1976|   9600|
|         2|   223500|     2001|  11250|
|         3|   140000|     1915|   9550|
|         3|   250000|     2000|  14260|
|         2|   143000|     1993|  14115|
|         2|   307000|     2004|  10084|
|         2|   200000|     1973|  10382|
|         2|   129900|     1931|   6120|
|         1|   118000|     1939|   7420|
|         1|   129500|     1965|  11200|
|         3|   345000|     2005|  11924|
|         1|   144000|     1962|  12968|
|         3|   279500|     2006|  10652|
|         1|   157000|     1960|  10920|
|         2|   132000|     1929|   6120|
|         2|   149000|     1970|  11241|
|         2|    90000|     1967|  10791|
|         2|   159000|     2004|  13695|
|         1|   139000|     1958|   7560|
+----------+---------+---------+-------+
only showing top

In [31]:
from pyspark.sql.functions import *
convert_to_m2 = 0.092093

df_final = df.select(col("GarageCars").alias("garagem"), \
               col("SalePrice").alias("preco"), \
               col("YearBuilt").alias("ano"), \
               round((col("LotArea") * convert_to_m2), 1).alias("tamanho"))
df_final.show()

+-------+------+----+-------+
|garagem| preco| ano|tamanho|
+-------+------+----+-------+
|      2|208500|2003|  778.2|
|      2|181500|1976|  884.1|
|      2|223500|2001| 1036.0|
|      3|140000|1915|  879.5|
|      3|250000|2000| 1313.2|
|      2|143000|1993| 1299.9|
|      2|307000|2004|  928.7|
|      2|200000|1973|  956.1|
|      2|129900|1931|  563.6|
|      1|118000|1939|  683.3|
|      1|129500|1965| 1031.4|
|      3|345000|2005| 1098.1|
|      1|144000|1962| 1194.3|
|      3|279500|2006|  981.0|
|      1|157000|1960| 1005.7|
|      2|132000|1929|  563.6|
|      2|149000|1970| 1035.2|
|      2| 90000|1967|  993.8|
|      2|159000|2004| 1261.2|
|      1|139000|1958|  696.2|
+-------+------+----+-------+
only showing top 20 rows

